In [ ]:
# Importing libraries
import numpy as np
import pandas as pd
import math
import os

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
import nltk
import numpy as np
import gensim
from gensim.models import Word2Vec
import gensim.downloader as api
from nltk.data import find
import matplotlib
import string
from urllib.request import urlopen
import nltk, re, pprint
from nltk import word_tokenize
from nltk import tokenize

## Vizualise
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
#!unzip yelp_reviews.zip
print('Dataset Downloaded: n_movies.csv')
df=pd.read_csv('n_movies.csv')
print(df.head())

print('Setup Complete')

Dataset Downloaded: n_movies.csv
                    title         year  length of characters certificate  \
0               Cobra Kai     (2018– )                    10       TV-14   
1               The Crown     (2016– )                    10       TV-MA   
2        Better Call Saul  (2015–2022)                    13       TV-MA   
3           Devil in Ohio        -2022                     5       TV-MA   
4  Cyberpunk: Edgerunners     (2022– )                    10       TV-MA   

  duration                         genre  rating  \
0   30 min         Action, Comedy, Drama     8.5   
1   58 min     Biography, Drama, History     8.7   
2   46 min                  Crime, Drama     8.9   
3  356 min        Drama, Horror, Mystery     5.9   
4   24 min  Animation, Action, Adventure     8.6   

                                         description  \
0  Decades after their 1984 All Valley Karate Tou...   
1  Follows the political rivalries and romance of...   
2  The trials and tribulation

In [ ]:
df.head(10)
# shows have -
# movies don't have -

,title,year,length of characters,certificate,duration,genre,rating,description,stars,votes
0,Cobra Kai,(2018– ),10,TV-14,30 min,"Action, Comedy, Drama",8.5,Decades after their 1984 All Valley Karate Tou...,"['Ralph Macchio, ', 'William Zabka, ', 'Courtn...",177031.0
1,The Crown,(2016– ),10,TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of...,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...",199885.0
2,Better Call Saul,(2015–2022),13,TV-MA,46 min,"Crime, Drama",8.9,The trials and tribulations of criminal lawyer...,"['Bob Odenkirk, ', 'Rhea Seehorn, ', 'Jonathan...",501384.0
3,Devil in Ohio,-2022,5,TV-MA,356 min,"Drama, Horror, Mystery",5.9,When a psychiatrist shelters a mysterious cult...,"['Emily Deschanel, ', 'Sam Jaeger, ', 'Gerardo...",9773.0
4,Cyberpunk: Edgerunners,(2022– ),10,TV-MA,24 min,"Animation, Action, Adventure",8.6,A Street Kid trying to survive in a technology...,"['Zach Aguilar, ', 'Kenichiro Ohashi, ', 'Emi ...",15413.0
5,The Sandman,(2022– ),10,TV-MA,45 min,"Drama, Fantasy, Horror",7.8,Upon escaping after decades of imprisonment by...,"['Tom Sturridge, ', 'Boyd Holbrook, ', 'Patton...",116358.0
6,Rick and Morty,(2013– ),10,TV-MA,23 min,"Animation, Adventure, Comedy",9.2,An animated series that follows the exploits o...,"['Justin Roiland, ', 'Chris Parnell, ', 'Spenc...",502160.0
7,Breaking Bad,(2008–2013),13,TV-MA,49 min,"Crime, Drama, Thriller",9.5,A high school chemistry teacher diagnosed with...,"['Bryan Cranston, ', 'Aaron Paul, ', 'Anna Gun...",1831340.0
8,The Imperfects,(2022– ),10,TV-MA,45 min,"Action, Adventure, Drama",6.3,After an experimental gene therapy turns them ...,"['Morgan Taylor Campbell, ', 'Italia Ricci, ',...",3123.0
9,Blonde,-2022,5,NC-17,166 min,"Biography, Drama, Mystery",6.2,A fictionalized chronicle of the inner life of...,"['Andrew Dominik', '| ', ' Stars:', 'Ana de...",935.0


In [ ]:
df = df.dropna(subset= ['year'])

In [ ]:
# to analyze shows recommendation system only going to look at shows, subset year to only columns with dash
df
df['year_length'] = df['year'].astype(str).len()

AttributeError: ignored

In [ ]:
df.columns

Index(['title', 'year', 'certificate', 'duration', 'genre', 'rating',
       'description', 'stars', 'votes'],
      dtype='object')

In [ ]:
df.shape

(9957, 10)

In [ ]:
df.isna().sum() 
# removing NA bc not a significant amount compared to total

title                     0
year                      0
length of characters      0
certificate             660
duration                122
genre                     0
rating                    0
description               0
stars                     0
votes                     0
dtype: int64

In [ ]:
df[["title", "genre","description"]]

,title,genre,description
0,Cobra Kai,"Action, Comedy, Drama",Decades after their 1984 All Valley Karate Tou...
1,The Crown,"Biography, Drama, History",Follows the political rivalries and romance of...
2,Better Call Saul,"Crime, Drama",The trials and tribulations of criminal lawyer...
3,Devil in Ohio,"Drama, Horror, Mystery",When a psychiatrist shelters a mysterious cult...
4,Cyberpunk: Edgerunners,"Animation, Action, Adventure",A Street Kid trying to survive in a technology...
...,...,...,...
9952,The Imperfects,"Action, Adventure, Drama",After an experimental gene therapy turns them ...
9953,The Walking Dead,"Drama, Horror, Thriller",Sheriff Deputy Rick Grimes wakes up from a com...
9954,The Crown,"Biography, Drama, History",Follows the political rivalries and romance of...
9955,Supernatural,"Drama, Fantasy, Horror",Two brothers follow their father's footsteps a...


In [ ]:
df.head(2)

,title,year,length of characters,certificate,duration,genre,rating,description,stars,votes
0,Cobra Kai,(2018– ),10,TV-14,30 min,"Action, Comedy, Drama",8.5,Decades after their 1984 All Valley Karate Tou...,"['Ralph Macchio, ', 'William Zabka, ', 'Courtn...",177031.0
1,The Crown,(2016– ),10,TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of...,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...",199885.0


In [ ]:
df['votes'] = df['votes'].astype(str).astype(int)

ValueError: ignored

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9957 entries, 0 to 9956
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   title                 9957 non-null   object 
 1   year                  9430 non-null   object 
 2   length of characters  9957 non-null   int64  
 3   certificate           6504 non-null   object 
 4   duration              7921 non-null   object 
 5   genre                 9884 non-null   object 
 6   rating                8784 non-null   float64
 7   description           9957 non-null   object 
 8   stars                 9957 non-null   object 
 9   votes                 8784 non-null   float64
dtypes: float64(2), int64(1), object(7)
memory usage: 778.0+ KB


In [ ]:
df['rating'].describe()

count    4330.000000
mean        7.030901
std         0.970189
min         5.100000
25%         6.300000
50%         7.100000
75%         7.700000
max         9.900000
Name: rating, dtype: float64

In [ ]:
df['votes'].describe()

count    4.330000e+03
mean     3.857541e+04
std      1.217516e+05
min      1.001000e+03
25%      2.162250e+03
50%      4.999000e+03
75%      1.986700e+04
max      1.844075e+06
Name: votes, dtype: float64

In [ ]:
df=df[(df['rating'] > 5) & (df['votes'] > 1000) ]

In [ ]:
df["certificate"].unique()

array(['TV-14', 'TV-MA', 'PG-13', 'TV-PG', 'R', 'PG', 'Not Rated',
       'TV-Y7-FV', 'Approved', 'G', 'Unrated', 'TV-G', 'TV-Y7', 'TV-Y',
       'Passed', 'NC-17', '12'], dtype=object)

In [ ]:
children = ['TV-PG','PG','G','TV-G', 'TV-Y7', 'TV-Y']
young_adult = ['TV-14','PG-13','NC-17', '12']
adult = ['TV-MA','R','Not Rated', 'Approved','Unrated']


def genre_update(df, x):
  if df[x] in children:
    df['update_certificate'] = 'children'
  else:
    df['update_certificate'] = 'update'


In [ ]:
df_test = df.copy()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

(3572, 10)

In [ ]:
df['certificate'] = df['certificate'].values.tolist()
df['genre'] = df['genre'].values.tolist()
df['description'] = df['description'].values.tolist()
df

,title,year,length of characters,certificate,duration,genre,rating,description,stars,votes
0,Cobra Kai,(2018– ),10,[TV-14],30 min,"[Action, Comedy, Drama]",8.5,"[Decades, after, their, 1984, All, Valley, Kar...","['Ralph Macchio, ', 'William Zabka, ', 'Courtn...",177031.0
1,The Crown,(2016– ),10,[TV-MA],58 min,"[Biography, Drama, History]",8.7,"[Follows, the, political, rivalries, and, roma...","['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...",199885.0
2,Better Call Saul,(2015–2022),13,[TV-MA],46 min,"[Crime, Drama]",8.9,"[The, trials, and, tribulations, of, criminal,...","['Bob Odenkirk, ', 'Rhea Seehorn, ', 'Jonathan...",501384.0
3,Devil in Ohio,-2022,5,[TV-MA],356 min,"[Drama, Horror, Mystery]",5.9,"[When, a, psychiatrist, shelters, a, mysteriou...","['Emily Deschanel, ', 'Sam Jaeger, ', 'Gerardo...",9773.0
4,Cyberpunk: Edgerunners,(2022– ),10,[TV-MA],24 min,"[Animation, Action, Adventure]",8.6,"[A, Street, Kid, trying, to, survive, in, a, t...","['Zach Aguilar, ', 'Kenichiro Ohashi, ', 'Emi ...",15413.0
...,...,...,...,...,...,...,...,...,...,...
9952,The Imperfects,(2022– ),10,[TV-MA],45 min,"[Action, Adventure, Drama]",6.3,"[After, an, experimental, gene, therapy, turns...","['Morgan Taylor Campbell, ', 'Italia Ricci, ',...",3130.0
9953,The Walking Dead,(2010–2022),13,[TV-MA],44 min,"[Drama, Horror, Thriller]",8.1,"[Sheriff, Deputy, Rick, Grimes, wakes, up, fro...","['Andrew Lincoln, ', 'Norman Reedus, ', 'Melis...",970067.0
9954,The Crown,(2016– ),10,[TV-MA],58 min,"[Biography, Drama, History]",8.7,"[Follows, the, political, rivalries, and, roma...","['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...",199898.0
9955,Supernatural,(2005–2020),13,[TV-14],44 min,"[Drama, Fantasy, Horror]",8.4,"[Two, brothers, follow, their, father's, foots...","['Jared Padalecki, ', 'Jensen Ackles, ', 'Jim ...",439601.0


In [ ]:
def split_cast(data):
  split_data=data.split(", ', '")
  names=[]
  for i in split_data:
    if '[' in i or ']' in i:
        i=i.replace('[','')
        i=i.replace(']','')
        i=i.replace("'",'')
        i=i.replace(' ','')
        names.append(i)
    else:
      i=i.replace(' ','')
      names.append(i)
  if type(names)==list:
    return names
  else :
    return list(names)

In [ ]:
df['stars'] = df['stars'].apply(split_cast)

In [ ]:
f = df['stars'].split(", ', '")

In [ ]:
df[df["title"]=="Shutter"]

,title,year,length of characters,certificate,duration,genre,rating,description,stars,votes,combined_description
1321,Shutter,(II) (2004),11,Not Rated,97 min,"Horror, Mystery, Thriller",7.0,A young photographer and his girlfriend discov...,"['Banjong Pisanthanakun, ', 'Parkpoom Wongpoom...",45222.0,"Horror, Mystery, Thriller A young photographer..."


In [ ]:
df_test['certificate'] = df['certificate'].values.tolist()
df_test['genre'] = df['genre'].values.tolist()
df_test['description'] = df['description'].values.tolist()
df_test

,title,year,length of characters,certificate,duration,genre,rating,description,stars,votes
0,Cobra Kai,(2018– ),10,[TV-14],30 min,"[Action, Comedy, Drama]",8.5,"[Decades, after, their, 1984, All, Valley, Kar...","['Ralph Macchio, ', 'William Zabka, ', 'Courtn...",177031.0
1,The Crown,(2016– ),10,[TV-MA],58 min,"[Biography, Drama, History]",8.7,"[Follows, the, political, rivalries, and, roma...","['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...",199885.0
2,Better Call Saul,(2015–2022),13,[TV-MA],46 min,"[Crime, Drama]",8.9,"[The, trials, and, tribulations, of, criminal,...","['Bob Odenkirk, ', 'Rhea Seehorn, ', 'Jonathan...",501384.0
3,Devil in Ohio,-2022,5,[TV-MA],356 min,"[Drama, Horror, Mystery]",5.9,"[When, a, psychiatrist, shelters, a, mysteriou...","['Emily Deschanel, ', 'Sam Jaeger, ', 'Gerardo...",9773.0
4,Cyberpunk: Edgerunners,(2022– ),10,[TV-MA],24 min,"[Animation, Action, Adventure]",8.6,"[A, Street, Kid, trying, to, survive, in, a, t...","['Zach Aguilar, ', 'Kenichiro Ohashi, ', 'Emi ...",15413.0
...,...,...,...,...,...,...,...,...,...,...
9952,The Imperfects,(2022– ),10,[TV-MA],45 min,"[Action, Adventure, Drama]",6.3,"[After, an, experimental, gene, therapy, turns...","['Morgan Taylor Campbell, ', 'Italia Ricci, ',...",3130.0
9953,The Walking Dead,(2010–2022),13,[TV-MA],44 min,"[Drama, Horror, Thriller]",8.1,"[Sheriff, Deputy, Rick, Grimes, wakes, up, fro...","['Andrew Lincoln, ', 'Norman Reedus, ', 'Melis...",970067.0
9954,The Crown,(2016– ),10,[TV-MA],58 min,"[Biography, Drama, History]",8.7,"[Follows, the, political, rivalries, and, roma...","['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...",199898.0
9955,Supernatural,(2005–2020),13,[TV-14],44 min,"[Drama, Fantasy, Horror]",8.4,"[Two, brothers, follow, their, father's, foots...","['Jared Padalecki, ', 'Jensen Ackles, ', 'Jim ...",439601.0


In [ ]:
df

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3572 entries, 0 to 9956
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   title                 3572 non-null   object 
 1   year                  3572 non-null   object 
 2   length of characters  3572 non-null   int64  
 3   certificate           3572 non-null   object 
 4   duration              3572 non-null   object 
 5   genre                 3572 non-null   object 
 6   rating                3572 non-null   float64
 7   description           3572 non-null   object 
 8   stars                 3572 non-null   object 
 9   votes                 3572 non-null   float64
dtypes: float64(2), int64(1), object(7)
memory usage: 307.0+ KB


In [ ]:
# trying without space in combined description
# previously reading as TV-14Action
df["combined_description"] = df['genre'] + ' ' + df['description']+ ' ' + df['stars']

In [ ]:
print(df_final[df_final['title'] == "Jumanji: Welcome to the Jungle"]['combined_description'])

320    action adventure comedy four teenagers are suc...
Name: combined_description, dtype: object


In [ ]:
# Escape Room Versailles top 10 when using certificate
Jumanji: Welcome to the Jungle
Jumanji: The Next Level
Shutter
War of the Worlds: Goliath
The Devil Inside
Insidious: Chapter 2
Noah
The Ridiculous 6
Trollhunters: Rise of the Titans


# without certificate:
The Highwaymen
Borgia
Enrico Piaggio: Vespa
John Was Trying to Contact Aliens
The Real Miyagi
Trailer Park Boys: Don't Legalize It
Oh, Hello on Broadway
Mongol: The Rise of Genghis Khan
Coco Before Chanel
LBJ

In [ ]:
df

In [ ]:
df_final = df[['title', 'combined_description']]

In [ ]:
df_final

,title,combined_description
0,Cobra Kai,"Action, Comedy, Drama Decades after their 1984..."
1,The Crown,"Biography, Drama, History Follows the politica..."
2,Better Call Saul,"Crime, Drama The trials and tribulations of cr..."
3,Devil in Ohio,"Drama, Horror, Mystery When a psychiatrist she..."
4,Cyberpunk: Edgerunners,"Animation, Action, Adventure A Street Kid tryi..."
...,...,...
9952,The Imperfects,"Action, Adventure, Drama After an experimental..."
9953,The Walking Dead,"Drama, Horror, Thriller Sheriff Deputy Rick Gr..."
9954,The Crown,"Biography, Drama, History Follows the politica..."
9955,Supernatural,"Drama, Fantasy, Horror Two brothers follow the..."


In [ ]:
# remove comma 

In [ ]:
from string import punctuation
p_list=list(punctuation)
p_list

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~']

In [ ]:
def punc(df):
  df = ''.join(df)
  for p in p_list:
    df=df.replace(p,'')
  return df.lower()

In [ ]:
# First round, noticed duplicates so now deleting duplicates
df_final.drop_duplicates("title",inplace=True)
df_final.dropna(inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
df_final['combined_description'] = df_final['combined_description'].apply(punc)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
cv= CountVectorizer(stop_words='english',max_features=8000)

In [ ]:
vector_netflix=cv.fit_transform(df_final['combined_description']).toarray()

In [ ]:
similarity=cosine_similarity(vector_netflix)

In [ ]:
test_similar = df_final[df_final["title"] == 'Versailles'].index[0]
test_similar

630

In [ ]:
print(df_final[df_final['title']=='Versailles'].index[0])

sorted(enumerate(similarity[630]),reverse=True,key=lambda x :x[1])[1:11]

630


[(969, 0.4472135954999579),
 (1217, 0.28867513459481287),
 (3247, 0.26352313834736496),
 (5812, 0.25425669046549126),
 (3764, 0.25318484177091666),
 (4587, 0.24152294576982392),
 (4591, 0.23732222626728364),
 (2249, 0.2357022603955158),
 (2475, 0.2357022603955158),
 (2416, 0.23354968324845687)]

In [ ]:
test = sorted(enumerate(similarity[630]),reverse=True,key=lambda x :x[1])[1:11]

for x in test:
  test = df_final.iloc[x[0]]
  print(test)

title                                                      The Highwaymen
combined_description    biography crime drama the untold true story of...
Name: 981, dtype: object
title                                                              Borgia
combined_description    biography drama history story of the rise and ...
Name: 1231, dtype: object
title                                               Enrico Piaggio: Vespa
combined_description    biography drama romance the life of enrico pia...
Name: 3290, dtype: object
title                                   John Was Trying to Contact Aliens
combined_description    documentary short biography john shepherd spen...
Name: 5905, dtype: object
title                                                     The Real Miyagi
combined_description    documentary action biography the life of fumio...
Name: 3818, dtype: object
title                                Trailer Park Boys: Don't Legalize It
combined_description    comedy crime drama after his late

In [ ]:
test = sorted(enumerate(similarity[630]),reverse=True,key=lambda x :x[1])[1:11]

for x in test:
  test = df_final.iloc[x[0]]
  print(test[0])

The Highwaymen
Borgia
Enrico Piaggio: Vespa
John Was Trying to Contact Aliens
The Real Miyagi
Trailer Park Boys: Don't Legalize It
Oh, Hello on Broadway
Mongol: The Rise of Genghis Khan
Coco Before Chanel
LBJ


In [ ]:
def recommended_5(entertainment):
  entertainment_index=df_final[df_final['title']==entertainment].index[0]
  entertainment=sorted(enumerate(similarity[entertainment_index]),reverse=True,key=lambda x :x[1])[1:6]
  
  for i in entertainment:
      entertainments=df_final.iloc[i[0]]
      if len(entertainments)> 0:
          print (entertainments[0])

In [ ]:
df_final[df_final['title'] == "The Highwaymen"]['combined_description']

981    biography crime drama the untold true story of...
Name: combined_description, dtype: object

In [ ]:
df_final[df_final['title'] == "Versailles"]['combined_description']

630    biography drama history in 1667 28yearold allp...
Name: combined_description, dtype: object

In [ ]:
df_final[df_final['title'] == "Borgia"]['combined_description']

1231    biography drama history story of the rise and ...
Name: combined_description, dtype: object

In [ ]:
recommended_5('Versailles')

The Highwaymen
Borgia
Enrico Piaggio: Vespa
John Was Trying to Contact Aliens
The Real Miyagi


In [ ]:
recommended_5('Narcos')

The Infiltrator
Viva La Madness
2020
Cocaine Cowboys: The Kings of Miami
Goals Against


In [ ]:
recommended_5('Sherlock')

Line of Duty
Queen Sono
Murder, She Wrote
Those Who Read the Hearts of Evil
Delhi Crime


In [ ]:
df

In [ ]:
df_final[df_final['title'] == "Versailles"]['combined_description']

630    biography drama history in 1667 28yearold allp...
Name: combined_description, dtype: object

In [ ]:
df_final[df_final['title'] == "The Love Section"]['combined_description']

6672    comedy drama romance ali lawrence adisa is a s...
Name: combined_description, dtype: object

In [ ]:
recommended_5('Cobra Kai')

The Love Section
The Thundermans
Teachers
Assassin's Creed
Untitled Scott Pilgrim anime series


In [ ]:
# duplicates, need to remove

In [ ]:
df

In [ ]:
def split_cast(data):
  split_data=data.split(", ', '")
  names=[]
  for i in split_data:
    if '[' in i or ']' in i:
        i=i.replace('[','')
        i=i.replace(']','')
        i=i.replace("'",'')
        i=i.replace(' ','')
        names.append(i)
    else:
      i=i.replace(' ','')
      names.append(i)
  if type(names)==list:
    return names
  else :
    return list(names)

In [ ]:
df["genre_category"] = "update"

In [ ]:
children = ['TV-PG','PG','G','TV-G', 'TV-Y7', 'TV-Y']
young_adult = ['TV-14','PG-13','NC-17', '12']
adult = ['TV-MA','R','Not Rated', 'Approved','Unrated']

def genre_update(data):
  if data in children:
    df["genre_category "] = "children_genre"
  elif data in young_adult:
    df["genre_category "] = "young_adult_genre"
  else:
    df["genre_category "] = "adult_genre"
  return df["genre_category"]

In [ ]:
df["certificate"]

0       TV-14
1       TV-MA
2       TV-MA
3       TV-MA
4       TV-MA
        ...  
9952    TV-MA
9953    TV-MA
9954    TV-MA
9955    TV-14
9956    TV-MA
Name: certificate, Length: 9957, dtype: object

In [ ]:
genre_update(df["certificate"])